# Understanding All LangChain Runnable Types

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.runnables import (
    RunnableLambda,
    RunnableMap,
    RunnableSequence,
    RunnableBranch,
    RunnableParallel,
    RunnablePassthrough,
    RunnablePick
)
import random

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
api_key=os.getenv("GROQ_API_KEY")

python-dotenv could not parse statement starting at line 16


In [5]:

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
model=ChatGroq(model="openai/gpt-oss-20b",api_key=api_key)
parser=StrOutputParser()

## RunnableLambda — Wraps any Python function as a Runnable

In [4]:


def greet(name: str):
    return f"Hello, {name}! Welcome to LangChain Runnables."

r_lambda = RunnableLambda(greet)
print(r_lambda.invoke("Warrior's"))

Hello, Warrior's! Welcome to LangChain Runnables.


## RunnableMap — Runs multiple Runnables in parallel

In [6]:

prompt1 = ChatPromptTemplate.from_template("Summarize the topic: {topic}")
prompt2 = ChatPromptTemplate.from_template("Write a haiku about {topic}")

chain1 = prompt1 | model | parser
chain2 = prompt2 | model | parser

r_map = RunnableMap({
    "summary": chain1,
    "haiku": chain2
})

result = r_map.invoke({"topic": "artificial intelligence"})
print(result)



{'summary': '**Artificial Intelligence (AI) – A Quick Summary**\n\n| Area | Key Points |\n|------|------------|\n| **What It Is** | AI is the design of computer systems that can perform tasks normally requiring human intelligence: perception, reasoning, learning, language understanding, and decision‑making. |\n| **Core Branches** | 1. **Machine Learning (ML)** – Algorithms that learn from data. <br>2. **Deep Learning** – Neural networks with many layers that excel at vision, speech, and language. <br>3. **Natural Language Processing (NLP)** – Understanding and generating human language. <br>4. **Computer Vision** – Interpreting visual input. <br>5. **Robotics & Autonomous Systems** – Physical agents that act in the world. |\n| **Historical Milestones** | • 1956 Dartmouth Conference (birth of AI). <br>• 1960s–70s rule‑based expert systems. <br>• 1980s–90s resurgence of machine learning (e.g., neural nets, support vector machines). <br>• 2000s–present: Big data, GPUs, deep learning break

## RunnableSequence — Sequential chain of Runnables

In [7]:

prompt = ChatPromptTemplate.from_template("Define {term} in one line.")
seq = RunnableSequence(first=prompt, middle=[model], last=parser)
print(seq.invoke({"term": "machine learning"}))


Machine learning is the field of AI that designs algorithms enabling computers to automatically improve their performance on tasks by learning from data rather than being explicitly programmed.


## RunnableBranch — Executes conditional logic

In [15]:

def condition_fn(input):
    return input["number"] % 2 == 0


even_chain = RunnableLambda(lambda x: f"{x['number']} is even")
odd_chain = RunnableLambda(lambda x: f"{x['number']} is odd")


r_branch = RunnableBranch(
    (condition_fn, even_chain),
    odd_chain                  
)


print(r_branch.invoke({"number": 4}))  
print(r_branch.invoke({"number": 7}))  


4 is even
7 is odd


## RunnableParallel — Executes multiple branches concurrently

In [16]:

parallel = RunnableParallel({
    "uppercase": RunnableLambda(lambda x: x["text"].upper()),
    "reversed": RunnableLambda(lambda x: x["text"][::-1]),
    "length": RunnableLambda(lambda x: len(x["text"]))
})

print(parallel.invoke({"text": "LangChain"}))


{'uppercase': 'LANGCHAIN', 'reversed': 'niahCgnaL', 'length': 9}


## RunnablePassthrough — Returns input as output

In [17]:

passthrough = RunnablePassthrough()
print(passthrough.invoke({"msg": "No transformation happens here"}))


{'msg': 'No transformation happens here'}


## RunnablePick — Picks a specific field from the output

In [24]:

data_runnable = RunnableLambda(lambda _: {"name": "Warrior", "role": "AI Engineer", "country": "UK"})

pick_role = RunnablePick(keys="role")

print(pick_role.invoke(data_runnable.invoke(None))) 

AI Engineer


- Refer the below link for more information about types of runnables
- https://medium.com/@danushidk507/runnables-in-langchain-e6bfb7b9c0ca